In [38]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from itertools import combinations

from nets_algo import *

## Функция потерь

Создание функции потерь в torch
https://neptune.ai/blog/pytorch-loss-functions. Для задач бинарной классификации рекомендуют использовать CrossEntropy функцию потерь

In [2]:
loss_fn = nn.BCELoss()

# Загрузка и подговтовка данных

In [3]:
data = pd.read_csv(
    "/home/dranik/KFA/university/proc_2.csv", 
    index_col = 0
)

save_ind = data[data['Y'] == 0].sample(
    sum(data['Y']), random_state = 0
).index.union(data[data['Y'] == 1].index)
data = data.loc[save_ind]

Нужно провести One Hot Encoding

In [4]:
Y = np.array(data[['Y']])
X = data.drop('Y', axis = 1)

X = np.concatenate([
    OneHotEncoder(sparse = False).\
    fit_transform(X.loc[:,X.dtypes == "O"]),
    X.loc[:,X.dtypes != "O"].to_numpy()
], axis = 1)

Y.shape

(13926, 1)

Разбивка на Train/Test

In [5]:
X_train, X_test, y_train, y_test = \
    train_test_split(
        X,Y, random_state = 0, stratify = Y
)

## Создание набора данных и загрузчика данных

In [6]:
train_data = My_data_set(
    torch.tensor(X_train.astype('float32')), 
    torch.tensor(y_train.astype('float32'))
)

In [46]:
train_data_loader =\
torch.utils.data.DataLoader(
    train_data, batch_size=1000
)

# Различные варианты модели

## Модели с одним скрытым слоем

Опишем вещи общие для модлей этой группы

In [32]:
import os

sfn = "fiting_results2/"
auc_full = pd.DataFrame()

False

In [33]:
def save_folder_porecessor(result_name):
    if not result_name in os.listdir("fiting_results2"):
        os.mkdir("fiting_results2/" + result_name)

def model_info_save(
    hlr, epochs, file_name, 
    lc_plot_param = {},
    fit_params = {}
):
    learning_info, auc_info, nets = \
    model_fit_get_perfomance(
        hlr, epochs, loss_fn, 
        train_data_loader, X_test, y_test
    )
    
    save_folder_porecessor(file_name)
    learning_info.to_excel(sfn + file_name + '/lc.xlsx')
    
    global auc_full
    if any(auc_full.columns == file_name):
        auc_full.drop(
            file_name, inplace = True, axis = 1
        )
        
    auc_full = auc_full.join(
        pd.DataFrame(auc_info, columns = [file_name]),
        how = 'outer'
    )
    
    for col in learning_info:

        fig = plt.figure(figsize = [14, 7])
        plot_learning_curve(
            learning_info[col],
            **lc_plot_param
        )
        fig.savefig(
            sfn + file_name + "/" +\
            str(col) + '.png'
        )
        plt.close()
    
    return nets

### Обобщенный построитель модели с одним слоем

Базовый вариант

In [34]:
nets = model_info_save(
    [[i] for i in range(2, 20)],
    50, "one_hiden_initial"
)

[2] AUC = 0.6326271056431683
[3] AUC = 0.509450918665651
[4] AUC = 0.5107794875821531
[5] AUC = 0.5113482615608096
[6] AUC = 0.6452730890398508
[7] AUC = 0.6465020235354977
[8] AUC = 0.5737914955093579
[9] AUC = 0.6149027360205814
[10] AUC = 0.6811729874589296
[11] AUC = 0.6841537722020626
[12] AUC = 0.6771821670222604
[13] AUC = 0.6794242714068017
[14] AUC = 0.6558853095644754
[15] AUC = 0.6799951898349137
[16] AUC = 0.6454789561875779
[17] AUC = 0.6450335705314375
[18] AUC = 0.6756368437531033
[19] AUC = 0.6797926218401948


После анализа рисунков становиться понятно, что многие модели начинают блуждать гдето после 20 - 25 эпохи, хотя до того спуск проходит очень быстро - поднимем начальный шаг и вресте с тем увеличим скорость спада learning rate, кроме того сделаю побольше эпох. Кривые обучения буду строить начиная с 10 эпохи - чтобы начальный быстрый спуск не "съедал" масштабом колебания в конце.

In [35]:
nets = model_info_save(
    [[i] for i in range(2, 20)],
    100, "100ep_lr_0.5__lr_decr_1.5",
    fit_params = {'lr': 0.5, 'lr_decr': 1.5},
    lc_plot_param = {'start_show': 10}
)

[2] AUC = 0.5113103213012123
[3] AUC = 0.6406141901189708
[4] AUC = 0.6461081046662891
[5] AUC = 0.647122264301086
[6] AUC = 0.6456853182082565
[7] AUC = 0.683021832804864
[8] AUC = 0.634139932255192
[9] AUC = 0.6827735715409783
[10] AUC = 0.6201416260403466
[11] AUC = 0.6836798488723991
[12] AUC = 0.6833997507819818
[13] AUC = 0.6475181626620998
[14] AUC = 0.6874106960520027
[15] AUC = 0.680930664670459
[16] AUC = 0.6831348287954033
[17] AUC = 0.6835270980881079
[18] AUC = 0.6828593495191979
[19] AUC = 0.6825203615475799


In [36]:
nets = model_info_save(
    [[i] for i in range(2, 20)],
    100, "100ep_lr_1__lr_decr_2",
    fit_params = {'lr': 1, 'lr_decr': 2},
    lc_plot_param = {'start_show': 10}
)

[2] AUC = 0.5115937185446381
[3] AUC = 0.5107715696149328
[4] AUC = 0.6834970757957309
[5] AUC = 0.6342827855804578
[6] AUC = 0.6459573333738029
[7] AUC = 0.645345010575435
[8] AUC = 0.6449708866242769
[9] AUC = 0.6836887565855219
[10] AUC = 0.6830830320931707
[11] AUC = 0.6460210070268659
[12] AUC = 0.6816650561301397
[13] AUC = 0.6825634154943401
[14] AUC = 0.682410664710049
[15] AUC = 0.6827943562049316
[16] AUC = 0.6806273075513324
[17] AUC = 0.6461914082797523
[18] AUC = 0.6320180819978087
[19] AUC = 0.643786820609545


In [ ]:
nets = model_info_save(
    [[i] for i in range(7, 23)],
    200, "200ep_lr_1__lr_decr_2___7_23",
    fit_params = {'lr': 1, 'lr_decr': 2},
    lc_plot_param = {'start_show': 10}
)

Для рваные графики в конце - побуждают увеличивать скорость спада learning rate

In [23]:
nets = model_info_save(
    [[i] for i in range(7, 23)],
    200, "200ep_lr_1__lr_decr_2.5___7_23",
    fit_params = {'lr': 1, 'lr_decr': 2.5},
    lc_plot_param = {'start_show': 10}
)

[7] AUC = 0.5149473075777256
[8] AUC = 0.6836711061169266
[9] AUC = 0.6719421222989422
[10] AUC = 0.6291728924433231
[11] AUC = 0.6741264915058357
[12] AUC = 0.6642448684149318
[13] AUC = 0.659913740345441
[14] AUC = 0.6335950771358468
[15] AUC = 0.6191004133508805
[16] AUC = 0.685074070933769
[17] AUC = 0.6863369867054032
[18] AUC = 0.6754242133417089
[19] AUC = 0.6459553538819979
[20] AUC = 0.6328930173756493
[21] AUC = 0.5365570896983618
[22] AUC = 0.6849589304937744


В целом данные меры уже почти не помокают взрастить AUC, потому предпримем последние попытки для самых перспективный моделей 16 и 22 

In [29]:
nets = model_info_save(
    [[16], [22]],
    500, "500ep_lr_2__lr_decr_4___16,22",
    fit_params = {'lr': 2, 'lr_decr': 3},
    lc_plot_param = {'start_show': 300}
)

[16] AUC = 0.6416892191267736
[22] AUC = 0.5161825104640884


In [30]:
auc_full.to_excel("fiting_results2/full_aucs.xlsx")

## Модели с двумя скрытыми слоями

In [53]:
auc_full = pd.DataFrame()

Варианты числа нейронов в крытых слоях

In [51]:
neurs = [
    list(comb) for comb in 
    list(combinations([2,2,3,3,4,4,5,5,6,6],2))
]

Базовые модели с двумя слоями

In [52]:
nets = model_info_save(
    neurs,
    50, "two_hiden_initial"
)

[2, 2] AUC = 0.6523354209273853
[2, 3] AUC = 0.6508971551733523
[2, 3] AUC = 0.6485463436971826
[2, 4] AUC = 0.6516813638434606
[2, 4] AUC = 0.6501139362491467
[2, 5] AUC = 0.6500911720933885
[2, 5] AUC = 0.633563075351665
[2, 6] AUC = 0.6686040392849943
[2, 6] AUC = 0.6360031289167132
[2, 3] AUC = 0.6470988403147259
[2, 3] AUC = 0.6685492733450541
[2, 4] AUC = 0.6522220950215452
[2, 4] AUC = 0.6399838869367067
[2, 5] AUC = 0.672958591340845
[2, 5] AUC = 0.620668995648747
[2, 6] AUC = 0.6481616624563976
[2, 6] AUC = 0.6689501204355806
[3, 3] AUC = 0.6790534466086521
[3, 4] AUC = 0.6737583060300929
[3, 4] AUC = 0.6553694869916047
[3, 5] AUC = 0.49856272399186957
[3, 5] AUC = 0.6287197537776128
[3, 6] AUC = 0.6667076861357383
[3, 6] AUC = 0.6767826395929374
[3, 4] AUC = 0.649994506910241
[3, 4] AUC = 0.6692242800505827
[3, 5] AUC = 0.6763563890242459
[3, 5] AUC = 0.6686703522604642
[3, 6] AUC = 0.6501076678584307
[3, 6] AUC = 0.6240436992610887
[4, 4] AUC = 0.6730465137685202
[4, 5] AUC 

In [ ]:
nets = model_info_save(
    neurs,
    200, "100ep_lr_0.5__lr_decr_1.5_two"
)